<a href="https://colab.research.google.com/github/rahulvudari/EmotionDetection/blob/master/EmotionRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#import utils
import os
%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
#from tensorflow.keras.utils import plot_model

from IPython.display import SVG, Image
#from livelossplot import PlotLossesTensorFlowKeras()
import tensorflow as tf
print("Tensorflow version:", tf.__version__)

Tensorflow version: 2.3.0


In [ ]:
!pip install kaggle

In [ ]:
import json
import os, zipfile

In [ ]:
!mkdir .kaggle
token = {"username":1'username',"key":"key"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)


In [ ]:

!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!chmod 600 /root/.kaggle/kaggle.json
!kaggle -v
!kaggle datasets download -d jonathanoheix/face-expression-recognition-dataset

- path is now set to: {/content}
Kaggle API 1.5.4
100% 120M/121M [00:03<00:00, 29.8MB/s]
100% 121M/121M [00:03<00:00, 38.0MB/s]


In [ ]:
path = '/content/{/content}/datasets/jonathanoheix/face-expression-recognition-dataset/face-expression-recognition-dataset.zip'

In [ ]:
local_zip = '/content/{/content}/datasets/jonathanoheix/face-expression-recognition-dataset/face-expression-recognition-dataset.zip'

zipref = zipfile.ZipFile(local_zip,'r')
zipref.extractall('/tmp')
zipref.close()

In [ ]:
base_dir = '/tmp/images/images'
train_dir = os.path.join(base_dir,'train')
test_dir = os.path.join(base_dir,'validation')

In [ ]:
img_size = 48
batch_size = 64
datagen_train = ImageDataGenerator(horizontal_flip=True)
train_generator = datagen_train.flow_from_directory(train_dir,
                                                    target_size = (img_size, img_size),
                                                    color_mode='grayscale',
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)
datagen_validation = ImageDataGenerator(horizontal_flip=True)
validation_generator = datagen_validation.flow_from_directory(test_dir,
                                                    target_size = (img_size, img_size),
                                                    color_mode='grayscale',
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=False)

Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


In [ ]:
model = Sequential()

#1 - conv

model.add(Conv2D(64,(3,3),padding='same',input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#2 - conv
model.add(Conv2D(128,(5,5),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#3 - conv
model.add(Conv2D(512,(3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
#4 - conv
model.add(Conv2D(512,(3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#flatten
model.add(Flatten())
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(7,activation='softmax'))

opt = Adam(lr=0.0005)

model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 64)        256       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 128)       204928    
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 128)       5

In [ ]:
epochs = 15
steps_per_epoch = train_generator.n//train_generator.batch_size
validation_steps = validation_generator.n//validation_generator.batch_size

checkpoint = ModelCheckpoint('model_weights.h5',monitor='val_accuracy',
                            save_weights_only=True,mode='max',verbose=1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=2,min_lr=0.00001,mode='auto')
callbacks = [checkpoint, reduce_lr]

history = model.fit(x=train_generator,
                   steps_per_epoch=steps_per_epoch,
                   epochs=epochs,
                   validation_data=validation_generator,
                   validation_steps=validation_steps,
                   callbacks = callbacks )

Epoch 1/15
450/450 [==============================] - ETA: 0s - loss: 1.8014 - accuracy: 0.3091
Epoch 00001: saving model to model_weights.h5
450/450 [==============================] - 1169s 3s/step - loss: 1.8014 - accuracy: 0.3091 - val_loss: 1.8905 - val_accuracy: 0.3405
Epoch 2/15
450/450 [==============================] - ETA: 0s - loss: 1.4772 - accuracy: 0.4306
Epoch 00002: saving model to model_weights.h5
450/450 [==============================] - 1179s 3s/step - loss: 1.4772 - accuracy: 0.4306 - val_loss: 1.3676 - val_accuracy: 0.4697
Epoch 3/15
450/450 [==============================] - ETA: 0s - loss: 1.3371 - accuracy: 0.4867
Epoch 00003: saving model to model_weights.h5
450/450 [==============================] - 1184s 3s/step - loss: 1.3371 - accuracy: 0.4867 - val_loss: 1.2660 - val_accuracy: 0.5148
Epoch 4/15
450/450 [==============================] - ETA: 0s - loss: 1.2489 - accuracy: 0.5215
Epoch 00004: saving model to model_weights.h5
450/450 [========================

In [ ]:
model_json = model.to_json()
with open("model.json","w") as json_file:
    json_file.write(model_json)

In [ ]:
!ls

'{'				       model.json	  sample_data
 camera.py			       model.py		  templates
 haarcascade_frontalface_default.xml   model_weights.h5   videoplayback.mp4
 main.py			       __pycache__
'{'				       model.json	  sample_data
 camera.py			       model.py		  templates
 haarcascade_frontalface_default.xml   model_weights.h5   videoplayback.mp4
 main.py			       __pycache__


In [ ]:
!pip install flask-ngrok

In [ ]:
!python main.py

2020-09-30 08:30:12.155643: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-09-30 08:30:13.704790: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2020-09-30 08:30:13.710840: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2000144999 Hz
2020-09-30 08:30:13.711083: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2e3aa00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-09-30 08:30:13.711119: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-09-30 08:30:13.713137: I tensorflow/stream_executor/platform/default/dso_loa